In [1]:
# 常用的引用和创建对象实体
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)

# Motivation

**Spark支持很多种输入输出源**

支持Hadoop MapRedcue所使用的InputFormat和OutputFormat接口访问数据，大部分文件格式和存储系统(S3,HDFS,Cassandra,HBase等)都支持这种接口

高层API可以更方便的使用这些接口

常见的三种数据源
1. 文件格式与文件系统
2. Spark SQL中的结构化数据源
3. 数据库与键值存储


# File Formats

Spark支持的一些常见格式：
- 文本: 非结构化，普通的文本文件，每行一条记录
- JSON：半结构化，常见的基于文本的格式，半结构化，大多数库都要求每行一条记录
- CSV: 结构化，非常常见的基于文本的格式，通常在电子表格应用中使用
- SequenceFiles: 结构化，一种用于键值对的数据的常见Hadoop文件格式
- Protocol buffers: 结构化，一种快速节约空间的跨语言格式
- 对象文件: 结构化 用来将Spark作业中的数据存储下来已让共享的代码读取，改变类的时候它会失效，因为它依赖于Java序列化

## Text Files

**将一个文本文件读取为RDD时，输入的每一行都会成为RDD的一个元素**

**也可以将多个文件一起读取为一个Pair RDD，其中键是文件名，值是文件内容**

### 读取文本文件

使用文件路径作为参数调用SparkContext中的textFile()函数，就可以读取一个文本文件，如果要控制分区的话，可以指定minPartitions.

如果多个输入文件以一个包含数据的所有部分的目录形式出现，可以用两种方式处理
1. 仍然使用textFile()函数，传递目录作为参数，这样会把各部分都读到RDD中
2. 如果文件足够小，那么可以使用SparkContext.wholeTextFiles()方法，该方法返回一个Pair RDD，其中键是文件名


In [2]:
Txt_RDD = sc.textFile("/home/colin/SparkExamples/bashrc")

### 保存文本文件

saveAsTextFile()方法接受一个路径，并将RDD中的内容都输入到路径对应的文件中。**Spark将传入的路径作为目录对待，会在那个目录下输出多个文件。这样，Spark就可以从多个节点上并行输出了，**还有一些其他的输出格式备用

In [3]:
# Examples
lines = sc.parallelize(["hello world","hi Worriors","for the horde, bool and lighting!!!"])
lines.saveAsTextFile("/home/colin/SparkExamples/OutputExamples")
# 会自动创建目录，在该目录下生成输出文件，和一个_SUCCESS文件？？

## JSON

读取JSON数据的最简单方式就是讲数据作文文本文件读取，然后使用JSON解析器对RDD中的值进行映射操作。也可以使用我们喜欢的JSON序列化库将数据转化为字符串，然后将其写出去。

[JSON入门教程](http://www.jianshu.com/p/041c766742b0)

[Python3 JSON数据解析](http://www.runoob.com/python3/python3-json.html)

### 读取JSON

- 将数据作为文本文件读取,然后对JSON数据进行解析，这样的方法可以在所有支持的编程语言中使用
- Python，Java,Scala中都有大量可用的JSON库，这里使用Python的内建库



In [6]:
import json
# Python 字典类型转换为 JSON 对象
data = {
    'no' : 1,
    'name' : 'Runoob',
    'url' : 'http://www.runoob.com'
}

json_str = json.dumps(data)
print ("Python 原始数据：", repr(data))
print ("JSON 对象：", json_str)

data2 = json.loads(json_str)
print ("data2['name']: ", data2['name'])
print ("data2['url']: ", data2['url'])

Python 原始数据： {'no': 1, 'name': 'Runoob', 'url': 'http://www.runoob.com'}
JSON 对象： {"no": 1, "name": "Runoob", "url": "http://www.runoob.com"}
data2['name']:  Runoob
data2['url']:  http://www.runoob.com


In [9]:
# 存储JSON文件
with open('/home/colin/SparkExamples/OutputExamples/data.json', 'w') as f:
    json.dump(data, f)
# 读取以保存的JSON文件
with open('/home/colin/SparkExamples/OutputExamples/data.json', 'r') as f:
    JSON = json.load(f)
print(JSON)

{'no': 1, 'name': 'Runoob', 'url': 'http://www.runoob.com'}


In [13]:
# 以文本文件的形式读取 
JSON_RDD = sc.textFile('/home/colin/SparkExamples/OutputExamples/data.json')
# 在Python中读取非结构化的JSON
DATA = JSON_RDD.map(lambda x:json.loads(x))
print(DATA.collect())

[{'no': 1, 'name': 'Runoob', 'url': 'http://www.runoob.com'}]


### 保存JSON

In [20]:
Filter = DATA.filter(lambda x:x["name"])
resFilter = Filter.map(lambda x:json.dumps(x))
resFilter.saveAsTextFile("/home/colin/SparkExamples/OutputExamples/Outdata.json")
# 过滤出需要的信息并保存，但是产生是目录，看来此函数需要其他参数来控制，且如果目录已经存在了，则会报错

## 逗号分隔值与制表符分隔值 -- 逗号分割值(CSV)文件

CSV文件每行都有固定数目的字段，字段间用逗号隔开，记录通常是一行一条，与JSON中的字段不一样的是，每条记录都有没相关联的字段名，只能等到对应的序号，常规做法是使用第一行中每列的值作为字段名。

### 读取CSV

类似JSON文件，先把文件当做普通文本文档进行读取，然后进行处理，Python中使用自带的csv库

#### CSV的所有数据字段均没有包含换行符，可以使用textFile()读取并解析数据

In [32]:
import csv
# import StringIO

def loadRecord(line):
    """解析一行csv记录"""
    Input = StringIO.StringIO(line)
    reader = csv.DictReader(Input,fieldnames)
    pass

def pri(line):
    print(line)

CSV_RDD = sc.textFile("/home/colin/SparkExamples/SparkTest.csv")
# CSV_RDD.collect()

#### 如果字段中有换行符，就需要完整读入每个文件，然后解析各段

如果文件很大，读取和解析的过程很可能会不幸的成为性能瓶颈。程序实例如下:

<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/6578592316db8844c42042bb8982fa0b5ae38bfb/ipynbFiles/Materials/example-5-15.jpg" />

### 保存CSV

和JSON类似 --> Page:70

<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/185a49c8e9713d987a96e67e065639a71904ba92/ipynbFiles/Materials/Example-5-18.jpg" />

## SequenceFile

Hadoop中常用的文件格式，之后可以详细学习，这里只列出简单的一些操作

介绍：如下
<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/f6365bd27e2124b214f7864a7a7530e7525d631e/ipynbFiles/Materials/ReadSequenceFile.jpg" />


<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/f6365bd27e2124b214f7864a7a7530e7525d631e/ipynbFiles/Materials/SaveSequenceFile.jpg" />

## 对象文件--> page 73


## Hadoop输入输出格式

除了Spark封装的格式之外，也可以与任何hadoop支持的格式交互。Spark支持新旧两套Hadoop文件API,提供了很大的灵活性

### 读取其他的Hadoop输入格式

<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/3225dc0fbc0aa0d2f1fddbbee2830ec81c48150c/ipynbFiles/Materials/Example-5-24.jpg" />


<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/3225dc0fbc0aa0d2f1fddbbee2830ec81c48150c/ipynbFiles/Materials/5CH_2_6.jpg" />


## 文件压缩 

大数据处理常常需要进行数据压缩来节省存储空间和网络传输开销 --page: 77

# 文件系统

Spark支持读写很多种文件系统，可以使用任何我们想要的文件格式

## 本地/常规文件系统

**Spark支持从本地文件系统中读取文件，不过它需要文件在集群中所有的节点下的相同路径下都可以找到**，否则，需要在驱动器程序中从本地读取该文件而无需使用整个集群，然后再进行分发，很慢，**所以，推荐把文件先放在HDFS,NFS,S3等共享文件系统上**

## Amazon S3 --page 78
## HDFS --page 79

# Spark SQL中的结构化数据

**Page 79**

## Apache Hive --page 80
## JSON --page 80

# 数据库

通过数据库提供的Hadoop连接器或者自定义的Spark连接器，Spark可以访问一些常用的数据库系统，下面是4种常见的连接器

## Java数据库连接 --page 81
## Cassandra --page 82
## HBase --page 84
## Elasticsearch --page 85
